## Programming Exercise Background

The following expansion gives an approximation to the exact value of π:

$$\pi(N) = \frac{4}{N}\sum_{i=1}^N\frac{1}{1 + \left(\frac{i - \frac{1}{2}}{N}\right)^2}$$

We can check this by hand like so...

$$\pi(1) = 4\frac{4}{5} = 3.2$$, $$\pi(2) = 4\left(\frac{16}{17}+\frac{16}{25}\right) = 3.162$$

It can be shown that the approximation continues to become more accurate as N is increased.

### Exercises

Note that you must use double-precision variables for ALL floating-point numbers.

### Exercise 1

Write a program in C, C++, Fortran or Java that computes an approximation to π using the above formula for the following values of N: 1, 2, 10, 50, 100, 500. For each value of N, print out the approximate value π(N) and the error err(N). The error is the difference between π(N) and the true value of π, ie err(N) = π(N) − π. As N increases the value of the error should decrease.

In [1]:
import numpy as np

In [2]:
pi_true = np.pi
print(pi_true)

3.141592653589793


In [3]:
def pi_approx(N):
    summation = 0
    for i in range(N):
        alpha  = (i - 0.5)/N       
        summation = summation + 1 / (1 + alpha**2)
 
    result = (4 / N)*summation
        
    return result

In [4]:
pi_approx(5)

3.4950161216478772

In [5]:
def pi_error(N):
    
    pi_err = np.abs(pi_true - pi_approx(N))

    return pi_err

pi_error(5)

0.35342346805808411

In [10]:
N = [1,2,10,50,100,500]

outputs_list = []

for input in N:
    outputs = pi_approx(input)
    outputs_list.append(pi_error(input))
    print("{} inputs approximates \N{GREEK SMALL LETTER PI} to be {} with an error of {}.".format(input, outputs, pi_error(input)))
    

1 inputs approximates π to be 3.2 with an error of 0.05840734641020706.
2 inputs approximates π to be 3.764705882352941 with an error of 0.623113228763148.
10 inputs approximates π to be 3.3311788072817965 with an error of 0.18958615369200338.
50 inputs approximates π to be 3.1812159878239283 with an error of 0.03962333423413522.
100 inputs approximates π to be 3.161499736951266 with an error of 0.019907083361472733.
500 inputs approximates π to be 3.145588976923137 with an error of 0.003996323333343987.


### Exercise 2

We now want to find out the minimum value of N that is required to give a value for π(N) that is accurate to some specified value. We will call this value Nmin. By computing π(N) for increasing values of N, calculate Nmin such that err(Nmin) < 10−6

As we know from Exercise 1, we have seen N = 500 only has an error of: 0.00399632333334 which is more than 0.00001 that is required.
With this knowledge, we do not need to test each iteration up to 500. However, we do not know the upper bound to which we will need to test. We could test up to 10,000 and still not reach the smallest N that gives pi to the required accuracy. Therefore we shall make an abituary guess of the upper bound, i.e. a guess of an N that would surely produce the desired acurrcy of 5 million (5000000)

This brute force search of comparing each N up to the upper bound is clearly ineffecient and Exercise 3 will address this issue. However, for the purpose of this task, below is an iterative search for pi accurate to an error < 0.00001

In [15]:
import time

upper_bound = 5000000

def iterative_search(upper_bound):

    for input in range(500, upper_bound):
        if input % 5000 == 0:
            print("Iteration at {}".format(input))
        if pi_error(input) < 0.00001:
            print("Got 'eeeem at {}".format(input))
            break
            
            
start = time.time()

# Careful, a large upper will take a long time.
iterative_search(upper_bound)

end = time.time()
print("Took {} s".format((end - start)))

Iteration at 5000
Iteration at 10000
Iteration at 15000
Iteration at 20000
Iteration at 25000
Iteration at 30000
Iteration at 35000
Iteration at 40000
Iteration at 45000
Iteration at 50000
Iteration at 55000
Iteration at 60000
Iteration at 65000
Iteration at 70000
Iteration at 75000
Iteration at 80000
Iteration at 85000
Iteration at 90000
Iteration at 95000
Iteration at 100000
Iteration at 105000
Iteration at 110000
Iteration at 115000
Iteration at 120000
Iteration at 125000
Iteration at 130000
Iteration at 135000
Iteration at 140000
Iteration at 145000
Iteration at 150000
Iteration at 155000
Iteration at 160000
Iteration at 165000
Iteration at 170000
Iteration at 175000
Iteration at 180000
Iteration at 185000
Iteration at 190000
Iteration at 195000
Iteration at 200000
Got 'eeeem at 200000
Took 7694.693068027496 s


In [13]:
print("pi approx = {}".format(pi_approx(200000)))
print("pi error = {}".format(pi_error(200000)))

pi approx = 3.1416026535668897
pi error = 9.999977096608603e-06


The output shows we have found the smallest N that gives pi to within an error of 0.00001. The time taken to achieve this is very long. The next exercise will look at other methods one could use to dramatically reduce compute time.

### Exercise 3

This way of computing Nmin is clearly inefficient. For example, if we require err(Nmin) < 10−6. and we calculate err(2) = 0.02, it is a waste of time to calculate err(3) as it is already obvious that Nmin is very much larger than 2!   Rewrite your program so that is uses a more efficient way to locate the minimum value of N. Your new method must produce exactly the same value for Nmin as before but should be faster. For example, you might try and reduce the number of times that you have to evaluate err(N). You should also tell us how much faster your new program is.

### My Solution

The iterative method shown in exercise 2 takes a long time because number of checks one needs to do for each input scales as N^3. Therefore, if we double the input, we are affecting the time taken to check the result by a factor of 100. So whatever we do, checking a large value of N will take time, but there is nothing to say we have to check every value of N up to the desired result. If we know N = 50 is nowhere even close to the answer we require, then perhaps we could try N = 500 as our next input instead of N = 51, 52, 53 etc.

The approached used here is a Binary Search method, which splits the range at which we are looking at into two, and then asks which side is the answer on. When it determines whether the answer is on the left or right, it then splits that new range by two again and asks the same question. The process is repeated until the range is of size one + one. Then from here an iterative search of the 2 values if done to determine which input gives our desrired result. This method runs with complexity nlogn which is considerably quicker than our previous approch. This can be seen in the timing metrics shown below.

In [16]:
import math
import numpy as np
import time

start = time.time()

pi_true = np.pi
print(pi_true)

def pi_error(N):

    pi_err = np.abs(pi_true - pi_approx(N))

    return pi_err

def pi_approx(N):
    summation = 0
    for i in range(N):
        alpha  = (i - 0.5)/N
        summation = summation + 1 / (1 + alpha**2)

    result = (4 / N)*summation

    return result

# First search
guesses = []
guess = 10

while pi_error(guess) > 0.00001:
    guesses = [guess]
    guess = guess * 2
    guesses.append(guess)
    print(guess)

print(guesses.pop())

upper_lim = guess
lower_lim = guesses.pop()
middle = math.floor(upper_lim - lower_lim / 2)

print("Upper - {},\nLower - {},\nMiddle - {}".format(upper_lim, lower_lim, middle))

# Interval search
while True:
    #Is middle point above or below desired answer?
    if pi_error(middle) < 0.00001:
        print("Above")
        upper_lim = middle
        # lower, no change
        diff = math.floor((upper_lim - lower_lim) / 2)
        middle = lower_lim + diff
    else:
        print("Below")
        #upper_lim stays the same
        lower_lim = middle
        diff = math.floor((upper_lim - lower_lim) / 2)
        middle = lower_lim + diff
    print("Upper - {},\nLower - {},\nMiddle - {}".format(upper_lim, lower_lim, middle))
    if (middle == lower_lim) or (middle == upper_lim):
        for input in range(lower_lim, upper_lim + 1):

            if pi_error(input) < 0.00001:
                print("Got 'eeeem at {}".format(input))
                break
        break
        # return middle
        
end = time.time()
print("Took {} s".format((end - start)))

3.141592653589793
20
40
80
160
320
640
1280
2560
5120
10240
20480
40960
81920
163840
327680
327680
Upper - 327680,
Lower - 163840,
Middle - 245760
Above
Upper - 245760,
Lower - 163840,
Middle - 204800
Above
Upper - 204800,
Lower - 163840,
Middle - 184320
Below
Upper - 204800,
Lower - 184320,
Middle - 194560
Below
Upper - 204800,
Lower - 194560,
Middle - 199680
Below
Upper - 204800,
Lower - 199680,
Middle - 202240
Above
Upper - 202240,
Lower - 199680,
Middle - 200960
Above
Upper - 200960,
Lower - 199680,
Middle - 200320
Above
Upper - 200320,
Lower - 199680,
Middle - 200000
Above
Upper - 200000,
Lower - 199680,
Middle - 199840
Below
Upper - 200000,
Lower - 199840,
Middle - 199920
Below
Upper - 200000,
Lower - 199920,
Middle - 199960
Below
Upper - 200000,
Lower - 199960,
Middle - 199980
Below
Upper - 200000,
Lower - 199980,
Middle - 199990
Below
Upper - 200000,
Lower - 199990,
Middle - 199995
Below
Upper - 200000,
Lower - 199995,
Middle - 199997
Below
Upper - 200000,
Lower - 199997,
Middl

As can be seen above, the time taken to achieve the desired result is considerably shorter than the iterative search method.

Since I did not know the true value (although I did I had to apprach this in a way where I would not know the upper bound)